In [1]:

from langchain.chat_models.gigachat import GigaChat

model = GigaChat(
    credentials="",
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
)

In [2]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [3]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Привет! Меня зоут Либерий!")],
    config=config,
)

response.content

NotImplementedError in RootListenersTracer.on_llm_end callback: NotImplementedError("Trying to load an object that doesn't implement serialization: {'lc': 1, 'type': 'not_implemented', 'id': ['gigachat', 'models', 'usage', 'Usage'], 'repr': 'Usage(prompt_tokens=20, completion_tokens=31, total_tokens=51)'}")


'Здравствуйте, Либерий! Я генеративная языковая модель, могу с вами пообщаться. Расскажите немного о себе.'

In [4]:
response = with_message_history.invoke(
    [HumanMessage(content="Как меня зовут?")],
    config=config,
)

response.content

NotImplementedError in RootListenersTracer.on_llm_end callback: NotImplementedError("Trying to load an object that doesn't implement serialization: {'lc': 1, 'type': 'not_implemented', 'id': ['gigachat', 'models', 'usage', 'Usage'], 'repr': 'Usage(prompt_tokens=16, completion_tokens=38, total_tokens=54)'}")


'Я не могу знать вашего имени, но, судя по данным моего последнего обновления, вас зовут Александр. Если это не так, пожалуйста, сообщите мне ваше имя.'

In [5]:
model.invoke(
    [
        HumanMessage(content="Привет! Меня зовут Вася"),
        AIMessage(
            content="Здравствуйте, Вася! Я генеративная языковая модель от Сбера. Готова ответить на ваши вопросы."
        ),
        HumanMessage(content="Как меня зовут?"),
    ]
)

AIMessage(content='Вас зовут Вася.', response_metadata={'token_usage': Usage(prompt_tokens=60, completion_tokens=8, total_tokens=68), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-7f720621-bfa8-4c82-a3b0-95458390834e-0')

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты личный помощник. Старайся как можно лучше помочь пользователю.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [7]:
response = chain.invoke({"messages": [HumanMessage(content="Привет! Меня зовут Вася")]})

response.content

'Здравствуйте, Вася! Рада знакомству. Как я могу вам помочь?'

In [8]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc5"}}
response = with_message_history.invoke(
    [HumanMessage(content="Привет! Меня зовут Кира")],
    config=config,
)

response.content

'Здравствуйте, Кира! Рада знакомству с вами.'

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Как меня зовут?")],
    config=config,
)

response.content

'Вас зовут Кира.'

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты — личный ассистент. Старайся как можно лучше помочь пользователю. Отвечай на все вопросы пользователя на следующем языке: {language}. Не называй своего имени.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [11]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content="Привет! Меня зовут Вася")],
        "language": "Английский",
    }
)

response.content

'Hello, I am your personal assistant. How can I help you?'

In [12]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [13]:
config = {"configurable": {"session_id": "abc15"}}
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Привет! Меня зовут Коля")],
        "language": "Английский",
    },
    config=config,
)

response.content

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Назови мое имя!")],
        "language": "Английский",
    },
    config=config,
)

response.content

'Your name is Николай.'

In [14]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="Ты — личный ассистент"),
    HumanMessage(content="Привет! Меня зовут Вася"),
    AIMessage(content="Привет!"),
    HumanMessage(content="Я люблю шоколадное мороженое"),
    AIMessage(content="Здорово"),
    HumanMessage(content="Сколько будет 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Спасибо"),
    AIMessage(content="Пожалуйста!"),
    HumanMessage(content="Тебе нравится наш разговор?"),
    AIMessage(content="Да!"),
]

trimmer.invoke(messages)

[SystemMessage(content='Ты — личный ассистент'),
 HumanMessage(content='Сколько будет 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='Спасибо'),
 AIMessage(content='Пожалуйста!'),
 HumanMessage(content='Тебе нравится наш разговор?'),
 AIMessage(content='Да!')]

In [15]:
config = {"configurable": {"session_id": "abc19"}}
for r in with_message_history.stream(
    {
        "messages": [
            HumanMessage(content="Привет! Я Вася. Расскажи историю из 100 слов")
        ],
        "language": "русский",
    },
    config=config,
):
    
    print(r.content, end="|")

Однажды в лесу родился маленький зайчик. Он был очень любопытный и постоянно бегал вокруг своей мамы, заглядывая ей в глаза. Однажды мама-зайчиха сказала ему: «Вася, не бегай так быстро, а то упадешь!» Но Вася не послушал её и продолжал бегать. И вот однажды он упал и сломал себе лапку|.| С| тех| пор| он| стал| очень| о|сто|рож|ным| и| больше| не| бе|гал| так| быстро|.||